In [1]:
# Quantum discrete Gaussian over x = [-1, 0, 1]
# Requires: qiskit

import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector


def f(u: float, T: float) -> np.ndarray:
    """
    Return a length-3 numpy array of probabilities [p(-1), p(0), p(1)]
    where the probabilities follow a discrete Gaussian with mean `u`
    and variance `T` over x = [-1, 0, 1].

    The function prepares a 2-qubit quantum state (using Initialize) whose
    measurement probabilities on |00>,|01>,|10> map to x = -1,0,1.

    Args:
        u: mean (float)
        T: variance (float), must be > 0

    Returns:
        numpy.ndarray of shape (3,) with probabilities in order [-1, 0, 1]
    """
    allowed = np.array([-1.0, 0.0, 1.0])

    if T <= 0:
        raise ValueError("variance T must be > 0")

    # classical discrete Gaussian (unnormalized)
    exps = np.exp(- (allowed - u)**2 / (2.0 * T))
    probs = exps / np.sum(exps)

    # prepare amplitudes for basis states |00>,|01>,|10>, leaving |11> zero
    amps = np.sqrt(probs)
    statevec = np.zeros(4, dtype=complex)
    statevec[0:3] = amps  # |00>->-1, |01>->0, |10>->1
    statevec = statevec / np.linalg.norm(statevec)

    qc = QuantumCircuit(2)
    qc.initialize(statevec, qc.qubits)

    sv = Statevector.from_instruction(qc)
    measured_probs = np.abs(sv.data)**2

    return np.array([measured_probs[0], measured_probs[1], measured_probs[2]])


# Example usage
if __name__ == "__main__":
    probs = f(0.2, 0.5)
    print("Probabilities [p(-1), p(0), p(1)]:", probs)


Probabilities [p(-1), p(0), p(1)]: [0.13734866 0.55697628 0.30567506]


In [3]:
#Test with T=1/3, u=(-0.4.0.4)*sqrt(1/3)
for u in np.linspace(-0.4, 0.4, 5) * np.sqrt(1/3):
    print(f"u={u:.3f}, probs={f(u, 1/3)}")
    #Recovered u,T from moments of probs:
    p = f(u, 1/3)
    mean = p[2] - p[0]
    var = p[0] + p[2] - mean**2
    print(f"  recovered mean={mean:.3f}, var={var:.3f}")  

u=-0.231, probs=[0.28639012 0.64196553 0.07164435]
  recovered mean=-0.215, var=0.312
u=-0.115, probs=[0.21414587 0.67874619 0.10710794]
  recovered mean=-0.107, var=0.310
u=0.000, probs=[0.15428077 0.69143845 0.15428077]
  recovered mean=0.000, var=0.309
u=0.115, probs=[0.10710794 0.67874619 0.21414587]
  recovered mean=0.107, var=0.310
u=0.231, probs=[0.07164435 0.64196553 0.28639012]
  recovered mean=0.215, var=0.312
